In [1]:
import os
import sys
import glob
import traceback
from datetime import datetime
import pandas as pd
from plotly.graph_objs import Scatter,Layout
import plotly
import plotly.offline as py
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
plotly.offline.init_notebook_mode(connected=True)
import plotly.express as px
import gezi
from gezi import tqdm 
import tensorflow as tf
from absl import flags
# FLAGS = flags.FLAGS
# flags.DEFINE_string('f', None, '')
# FLAGS(sys.argv)
from projects.feed.rank.src.evaluate import evaluate_df as eval
import pymp
from multiprocessing import Pool, Manager, cpu_count 
import qgrid

In [2]:
mark = 'video'
root = f'/search/odin/publicData/CloudS/chenghuige/rank/data/{mark}_hour_sgsapp_v1/exps/'
root = f'/home/gezi/tmp/rank/exps/{mark}/monitor'
# root = f'/home/gezi/tmp/rank/data/{mark}_hour_sgsapp_v1/exps/v20'
root = f'/home/gezi/tmp/rank/{mark}/exps/v25'
root = '/home/gezi/tmp/rank/exps/video/monitor'
day_ = 20191224
days = 100
day = gezi.DateTime(str(day_), day=True)

In [3]:
models = [os.path.basename(x) for x in glob.glob(f'{root}/*')]
# names = [
#   'dlrm',
#   'baseline',
# ]
models

['dlrm-din',
 'dlrm-din-ft',
 'dlrm-din-ft2',
 'dlrm',
 'dlrm-alltype',
 'dlrm-residual',
 'torch-dlrm',
 'test2',
 'test3',
 '1',
 '8',
 'dlrm-att-kw-all-refresh',
 'dlrm-att-kw-atv3',
 'dlrm-att-kw-v3',
 'dlrm-att-kw-all-refresh2',
 'dlrm-att-kw-all-atv4',
 'dlrm-att-kw-refresh']

In [4]:
dfs = Manager().list()
files = []
model_names = []
for model in models:
  files_ = glob.glob(f'{root}/{model}/infos_day/*/metrics_impression.csv')
  if not files_:
    files_ =  glob.glob(f'{root}/{model}/infos_day/*/metrics.csv')
  files += files_
  model_names += [model] * len(files_)
ps = min(len(files), cpu_count())
with pymp.Parallel(ps) as p:
  for i in tqdm(p.range(len(files))):
    try:
      df = pd.read_csv(files[i])
      df['model'] = model_names[i]
      dfs += [df]
    except Exception:
      pass
    
dfs = list(dfs)
print(len(dfs))

100%|██████████| 12/12 [00:01<00:00,  7.34it/s]

100%|██████████| 12/12 [00:01<00:00,  7.56it/s]

 92%|█████████▏| 12/13 [00:02<00:00,  5.40it/s]

100%|██████████| 13/13 [00:03<00:00,  5.05it/s]


504


In [5]:
df = pd.concat(dfs)

In [6]:
df[df.model=='base'].sort_values('date')[['date', 'group/ndcg_click']].tail()

,date,group/ndcg_click


In [7]:
# df = df[df.model.isin(['deeponly-udh-simple', 'deeponly-udh-simple-leaky'])]
# df = df[df.model.isin([x for x in models if 'torch' in x])]
df = df[~df.model.isin(['online'])]

In [8]:
df['sqrt/auc'] = (df['auc'] * df['click/time_auc']) ** 0.5
df['group/sqrt/auc'] =  (df['group/auc'] * df['group/click/time_auc']) ** 0.5
# df['quality/sqrt/auc'] = (df['quality/auc'] * df['quality/click/time_auc']) ** 0.5
# df['quality/group/sqrt/auc'] =  (df['quality/group/auc'] * df['quality/group/click/time_auc']) ** 0.5
df['rmse'] = 1. - df['mse'] ** 0.5
df['click/rmse'] = 1. - df['click/mse'] ** 0.5

In [9]:
cols = ['date', 'model','sqrt/auc', 'group/sqrt/auc', 'inv_rate']
if mark == 'video':
  cols += ['finish_auc', 'vtime_auc']
qgrid.show_grid(df[cols].sort_values(['date', 'group/sqrt/auc'], ascending=[False, False]))

AttributeError: module 'pandas.core' has no attribute 'index'

In [10]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:500000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [11]:
# df[['date', 'model', 'auc', 'group/auc', 'cold/auc', 'clickmids/auc',
#     'click/time_auc','group/click/time_auc',
#    ]].sort_values(['date', 'group/auc'], ascending=[False, False]).head(10)

In [12]:
# df[[
#     'date', 'model','quality/auc', 'quality/group/auc', 
#     'quality/click/time_auc', 'quality/group/click/time_auc'
# ]].sort_values(['date', 'quality/click/time_auc'], ascending=[False, False]).head(10)

In [13]:
# df[[
#       'date', 'model', 'group/click/top1_dur_rate', 'group/click/top3_dur_rate', 'group/top1_click_rate', 'group/top3_click_rate'
# ]].sort_values(['date', 'group/top3_click_rate'], ascending=[False, False]).head(10)

### online 是线上tfrecord落盘数据直接计算 base是指定的模型比如16尾号的验证 有可能并不是当时线上全量作用于abtest 4,5,6的模型 
### 这里的验证都是针对abtest 4,5,6 小写click表示只针对正样本 Click/表示点击塔输出结果的指标 Dur/表示时长塔

In [14]:
df[['model', 'hours', 'date']].head()

,model,hours,date
0,dlrm-din,22,20200529
0,dlrm-din,24,20200611
0,dlrm-din-ft2,22,20200529
0,dlrm,24,20200525
0,dlrm,24,20200607


In [15]:
df = df[df.hours >= 20]

In [16]:
# df = df[df.model.isin(['dlrm', 'baseline'])]

In [17]:
metrics = ['hours', 
            'stats/click_ratio',
            'stats/num_instances',
            'group/auc',
            'group/click/time_auc',
            'Click/group/auc',
            'Click/group/auc2',
            'Dur/group/click/time_auc',
#             'Dur/group/click/time_auc',
            'group/auc2',
            'group/top1_click', 
            'group/ndcg3_click',
            'group/top1_dur',
#            'group/sqrt/auc', 'sqrt/auc', 
#             'quality/group/click/time_auc', 
            'rmse', 'click/rmse', 
            'rcr_auc', 'vtime_auc', 'ptime_auc', 
#            'Click/group/auc',  'Click/group/auc2', 'Dur/group/click/time_auc',
            'Click/rcr_auc', 'Click/vtime_auc', 'Dur/rcr_auc', 'Dur/vtime_auc', 'Click/group/click/time_auc',
          ]
gezi.line(df, x='date', y=metrics, color='model')

In [18]:
metrics = [
           'group/click/time_auc',
           'group/auc',
           'group/auc/quality',
           'group/auc/cold',
           'group/auc/activity_-1',
           'group/auc/activity_0',
           'group/auc/activity_1',
           'group/auc/activity_2',
           'Click/group/auc',
           'Click/group/auc/activity_-1',
           'Click/group/auc/activity_0',
           'Click/group/auc/activity_1',
           'Click/group/auc/activity_2',
           'Click/group/auc/quality',
           'Click/group/auc/cold',
#            'quality/group/click/time_auc', 'quality/group/auc',
           'group/top1_click_rate', 'group/top3_click_rate',
           'group/click/top1_dur_rate', 'group/click/top3_dur_rate',
           'Dur/group/click/time_auc',
          ]
gezi.line(df, x='date', y=metrics, color='model')

In [ ]:
metrics = [
  'group/first_click_position',
  'group/last_click_position',
  'group/ndcg_click',
  'group/ndcg3_click',
  'group/ndcg3_dur',
  'group/ndcg_dur',
#   'group/ndcg7_click',
#   'group/ndcg14_click',
  'group/click/ndcg_dur',
  'group/click/ndcg3_dur',
#   'group/click/ndcg7_dur',
#   'group/click/ndcg14_dur',
#   'quality/group/ndcg_dur',
#   'quality/group/click/ndcg_dur',
#   'quality/group/click/ndcg3_dur',
]
gezi.line(df, x='date', y=metrics, color='model')